In [1]:
import numpy as np
import polars as pl
import pymc as pm
import arviz as az
import altair as alt
import hvplot.polars
from arviz import waic
import matplotlib.pyplot as plt
from pybaseball import batting_stats
import seaborn as sns; sns.set_context('notebook')
import warnings
import IPython
import plotly.express as px
warnings.filterwarnings("ignore", module="mkl_fft")
warnings.filterwarnings("ignore", module="matplotlib")
RANDOM_SEED = 20090425

In [26]:
mb = pl.read_excel("Data/mbRunValuesbyPitchType.xlsx",
                    schema_overrides={'Year': pl.Int64,
                                    'Team': pl.String,
                                    'Pitch Type': pl.String,
                                    'RV/100': pl.Float64,
                                    'Run Value': pl.Int64,
                                    'Pitches': pl.Int64,
                                    '%': pl.Float64,
                                    'PA': pl.Int64,
                                    'BA': pl.Float64,
                                    'SLG': pl.Float64,
                                    'wOBA': pl.Float64,
                                    'Whiff%': pl.Float64,
                                    'K%': pl.Float64,
                                    'PutAway %': pl.Float64,
                                    'xBA': pl.Float64,
                                    'xSLG': pl.Float64,
                                    'xwOBA': pl.Float64,
                                    'Hard Hit %': pl.Float64,
                                    'Name': pl.String}
).drop(pl.col("Team", "RV/100","Run Value")).rename({'%' : 'Pitches %','Year' : 'Season'}).with_columns(pl.col("BA").shift(1).over("Pitch Type").alias("BAy")).filter(
    pl.col("Pitch Type").is_in(mb.group_by("Pitch Type").count().filter(pl.col("count")  >3)["Pitch Type"])).drop_nulls()

/tmp/ipykernel_28143/3579132802.py:21: DeprecationWarning:

`GroupBy.count` is deprecated. It has been renamed to `len`.



In [27]:
#mb.filter(pl.col("Pitch Type") == "Sinker")
mb.group_by("Pitch Type").count().hvplot.bar(x = "Pitch Type")

/tmp/ipykernel_28143/419903352.py:2: DeprecationWarning:

`GroupBy.count` is deprecated. It has been renamed to `len`.



:Bars   [Pitch Type]   (count)

In [28]:
mb.filter(pl.col("Pitch Type").is_in(mb.group_by("Pitch Type").count().filter(pl.col("count")  >3)["Pitch Type"]))

/tmp/ipykernel_28143/3169978015.py:1: DeprecationWarning:

`GroupBy.count` is deprecated. It has been renamed to `len`.



Season,Pitch Type,Pitches,Pitches %,PA,BA,SLG,wOBA,Whiff%,K%,PutAway %,xBA,xSLG,xwOBA,Hard Hit %,Name,BAy
i64,str,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
2023,"""4-Seam Fastball""",859,31.7,208,0.302,0.634,0.439,10.7,11.5,10.7,0.296,0.631,0.434,54.7,"""Mookie Betts""",0.278
2023,"""Sinker""",534,19.7,136,0.366,0.589,0.469,11.4,14.7,18.9,0.329,0.533,0.42,49.5,"""Mookie Betts""",0.359
2023,"""Slider""",451,16.6,129,0.281,0.588,0.417,25.5,20.9,17.8,0.268,0.559,0.386,44.8,"""Mookie Betts""",0.314
2023,"""Cutter""",242,8.9,62,0.308,0.5,0.381,9.2,3.2,4.7,0.357,0.646,0.453,42.3,"""Mookie Betts""",0.433
2023,"""Changeup""",228,8.4,69,0.193,0.456,0.344,24.3,20.3,17.7,0.23,0.51,0.376,46.5,"""Mookie Betts""",0.163
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2017,"""Changeup""",231,8.2,58,0.286,0.408,0.351,26.0,8.6,8.1,0.254,0.362,0.322,20.0,"""Mookie Betts""",0.361
2017,"""Curveball""",230,8.2,46,0.289,0.467,0.372,15.0,17.4,10.8,0.23,0.292,0.235,37.8,"""Mookie Betts""",0.394
2017,"""Cutter""",197,7.0,55,0.226,0.396,0.297,20.0,12.7,14.9,0.269,0.429,0.31,37.0,"""Mookie Betts""",0.296


In [35]:

fig = px.histogram(mb.filter(pl.col("Pitch Type") == "Sinker"), x="BAy",nbins=30, color = "Pitch Type",marginal="rug",hover_data=mb['Name','Season','PA','Pitches','BA','SLG','Whiff%','Hard Hit %','BAy'].columns)
fig.show()

In [31]:
mb.filter(pl.col("Pitch Type") == "Slurve")

Season,Pitch Type,Pitches,Pitches %,PA,BA,SLG,wOBA,Whiff%,K%,PutAway %,xBA,xSLG,xwOBA,Hard Hit %,Name,BAy
i64,str,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64
2023,"""Slurve""",12,0.4,2,0.0,0.0,0.0,25.0,0.0,0.0,0.148,0.179,0.134,50.0,"""Mookie Betts""",0.0
2022,"""Slurve""",2,0.1,1,1.0,1.0,0.9,0.0,0.0,0.0,0.451,0.701,0.482,100.0,"""Mookie Betts""",0.0
2021,"""Slurve""",2,0.1,1,1.0,1.0,0.9,0.0,0.0,0.0,0.92,1.132,0.898,100.0,"""Mookie Betts""",1.0
2019,"""Slurve""",19,0.7,4,1.0,2.0,1.125,0.0,0.0,0.0,0.831,1.91,1.023,66.7,"""Mookie Betts""",1.0
2018,"""Slurve""",19,0.7,6,0.0,0.0,0.117,28.6,16.7,14.3,0.232,0.348,0.321,25.0,"""Mookie Betts""",1.0
2017,"""Slurve""",15,0.5,6,0.333,0.333,0.3,25.0,33.3,28.6,0.439,0.504,0.405,75.0,"""Mookie Betts""",0.0
